In [1]:
import numpy as np
import pandas as pd
import anndata as ad
from scipy.sparse import csr_matrix
from mudata import MuData
import mudata as md
from anndata import AnnData
import pyranges as pr
import bioframe as bf

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def set_coord(adata, range_df):
    adata.varm['coord'] = range_df.set_index(adata.var_names)

In [4]:
def subset_by_overlap(adata, granges):
    # coord = pr.PyRanges(adata.varm['coord'].reset_index())
    varm = adata.varm['coord'].copy()
    varm['idx'] = varm.index
    idx = bf.overlap(varm, granges, how='inner')['idx']
    return adata[:, idx]

In [5]:
def slice_granges(adata, chrom, start, end):
    idx = bf.select(adata.varm['coord'], f"{chrom}:{start}-{end}").index
    return adata[:, idx]

In [6]:
class RangeAnnData(AnnData):
    def set_coord(self, range_df):
        self.varm['coord'] = range_df.set_index(self.var_names)

    def subset_by_overlap(self, granges):
        varm = self.varm['coord'].copy()
        varm['idx'] = varm.index
        idx = bf.overlap(varm, granges, how='inner')['idx']
        return self[:, idx]

    def slice_granges(self, chrom, start, end):
        idx = bf.select(self.varm['coord'], f"{chrom}:{start}-{end}").index
        return self[:, idx]

In [7]:
counts = csr_matrix(np.random.poisson(1, size=(100, 1000)), dtype=np.float32)
exons, gr = pr.data.exons().df, pr.data.cpg().df
exons.columns = ['chrom', 'start', 'end'] + list(exons.columns[3:])
gr.columns = ['chrom', 'start', 'end']+ list(gr.columns[3:])

adata = RangeAnnData(counts)
adata.obs_names = [f"Cell_{i:d}" for i in range(adata.n_obs)]
adata.var_names = [f"Gene_{i:d}" for i in range(adata.n_vars)]
adata.set_coord(exons)

In [8]:
adata.slice_granges('chrX', 1, 1500000)

View of AnnData object with n_obs × n_vars = 100 × 11
    varm: 'coord'

In [9]:
adata.subset_by_overlap(gr).varm['coord']

,chrom,start,end,Name,Score,Strand
Gene_846,chrY,16941609,16942399,NM_014893_exon_4_0_chrY_16941610_f,0,+
Gene_860,chrY,155399,155536,NR_028057_exon_3_0_chrY_155400_f,0,+
Gene_984,chrY,244667,245252,NM_013239_exon_0_0_chrY_244668_r,0,-
Gene_858,chrY,1363220,1363354,NM_172246_exon_7_0_chrY_1363221_f,0,+
Gene_881,chrY,14533348,14533389,NR_033667_exon_4_0_chrY_14533349_r,0,-
...,...,...,...,...,...,...
Gene_692,chrX,150066994,150067289,NM_031462_exon_10_0_chrX_150066995_r,0,-
Gene_326,chrX,150565656,150565833,NM_001017980_exon_0_0_chrX_150565657_f,0,+
Gene_133,chrX,153046670,153046801,NM_014370_exon_1_0_chrX_153046671_f,0,+
Gene_819,chrX,153599240,153599729,NM_001456_exon_45_0_chrX_153599241_r,0,-


In [10]:
counts = csr_matrix(np.random.poisson(1, size=(100, 10000)), dtype=np.float32)
chipseq, gr = pr.data.chipseq().df, pr.data.cpg().df
chipseq.columns = ['chrom', 'start', 'end'] + list(chipseq.columns[3:])
gr.columns = ['chrom', 'start', 'end']+ list(gr.columns[3:])

bdata = RangeAnnData(counts)
bdata.obs_names = [f"Cell_{i:d}" for i in range(bdata.n_obs)]
bdata.var_names = [f"Gene_{i:d}" for i in range(bdata.n_vars)]
bdata.set_coord(chipseq)

In [11]:
mdata = MuData({"Exon": adata, "Chipseq": bdata})

In [12]:
MuData({k: mdata.mod[k].subset_by_overlap(gr) for k in mdata.mod.keys()})

MuData object with n_obs × n_vars = 100 × 80
  2 modalities
    Exon:	100 x 79
      varm:	'coord'
    Chipseq:	100 x 1
      varm:	'coord'

In [13]:
class RangeMuData(MuData):
    def set_coord(self, prange):
        self.varm['coord'] = prange.df.set_index(adata.var_names)

    def subset_by_overlap(self, granges):
        return MuData({k: mdata.mod[k].subset_by_overlap(granges) for k in mdata.mod.keys()})

    def slice_granges(self, chrom, start, end):
        return MuData({k: mdata.mod[k].slice_granges(chrom, start, end) for k in mdata.mod.keys()})


In [14]:
mdata = RangeMuData({"rna": adata, "epic": bdata})

In [15]:
mdata.subset_by_overlap(gr)

MuData object with n_obs × n_vars = 100 × 80
  2 modalities
    rna:	100 x 79
      varm:	'coord'
    epic:	100 x 1
      varm:	'coord'

In [16]:
mdata.slice_granges('chrX', 1, 10000000)

MuData object with n_obs × n_vars = 100 × 37
  2 modalities
    rna:	100 x 33
      varm:	'coord'
    epic:	100 x 4
      varm:	'coord'

In [17]:
mdata.mod['rna'].varm['coord']

,chrom,start,end,Name,Score,Strand
Gene_0,chrX,135721701,135721963,NR_038462_exon_0_0_chrX_135721702_f,0,+
Gene_1,chrX,135574120,135574598,NM_001727_exon_2_0_chrX_135574121_f,0,+
Gene_2,chrX,47868945,47869126,NM_205856_exon_4_0_chrX_47868946_f,0,+
Gene_3,chrX,77294333,77294480,NM_000052_exon_17_0_chrX_77294334_f,0,+
Gene_4,chrX,91090459,91091043,NM_001168360_exon_0_0_chrX_91090460_f,0,+
...,...,...,...,...,...,...
Gene_995,chrY,15591133,15591197,NR_047643_exon_27_0_chrY_15591134_r,0,-
Gene_996,chrY,15409586,15409728,NR_047633_exon_3_0_chrY_15409587_r,0,-
Gene_997,chrY,15478146,15478273,NR_047634_exon_18_0_chrY_15478147_r,0,-
Gene_998,chrY,15360258,15361762,NR_047601_exon_0_0_chrY_15360259_r,0,-


In [18]:
adata.slice_granges('chrX', 1, 10000000).X.mean()

1.0069696

In [44]:
bf.overlap(adata.varm['coord'], gr, how='inner')

,chrom,start,end,Name,Score,Strand,chrom_,start_,end_,CpG_
0,chrX,17879217,17879457,NM_001172739_exon_2_0_chrX_17879218_r,0,-,chrX,17879219,17879702,40
1,chrX,34147868,34150447,NM_203408_exon_0_0_chrX_34147869_r,0,-,chrX,34148740,34148957,25
2,chrX,34147868,34150447,NM_203408_exon_0_0_chrX_34147869_r,0,-,chrX,34149264,34149486,20
3,chrX,45016959,45017133,NM_176819_exon_2_0_chrX_45016960_r,0,-,chrX,45016977,45017178,20
4,chrX,47039278,47039436,NM_001204467_exon_9_0_chrX_47039279_f,0,+,chrX,47039345,47039571,18
...,...,...,...,...,...,...,...,...,...,...
74,chrY,15591393,15592550,NR_047610_exon_27_0_chrY_15591394_r,0,-,chrY,15591259,15591720,33
75,chrY,15591393,15592550,NR_047607_exon_29_0_chrY_15591394_r,0,-,chrY,15591259,15591720,33
76,chrY,15591393,15592550,NM_001258269_exon_29_0_chrY_15591394_r,0,-,chrY,15591259,15591720,33
77,chrY,15591393,15592550,NR_047599_exon_28_0_chrY_15591394_r,0,-,chrY,15591259,15591720,33


In [42]:
bf.overlap(exons, gr, how='inner')

,chrom,start,end,Name,Score,Strand,chrom_,start_,end_,CpG_
0,chrX,17879217,17879457,NM_001172739_exon_2_0_chrX_17879218_r,0,-,chrX,17879219,17879702,40
1,chrX,34147868,34150447,NM_203408_exon_0_0_chrX_34147869_r,0,-,chrX,34148740,34148957,25
2,chrX,34147868,34150447,NM_203408_exon_0_0_chrX_34147869_r,0,-,chrX,34149264,34149486,20
3,chrX,45016959,45017133,NM_176819_exon_2_0_chrX_45016960_r,0,-,chrX,45016977,45017178,20
4,chrX,47039278,47039436,NM_001204467_exon_9_0_chrX_47039279_f,0,+,chrX,47039345,47039571,18
...,...,...,...,...,...,...,...,...,...,...
74,chrY,15591393,15592550,NR_047610_exon_27_0_chrY_15591394_r,0,-,chrY,15591259,15591720,33
75,chrY,15591393,15592550,NR_047607_exon_29_0_chrY_15591394_r,0,-,chrY,15591259,15591720,33
76,chrY,15591393,15592550,NM_001258269_exon_29_0_chrY_15591394_r,0,-,chrY,15591259,15591720,33
77,chrY,15591393,15592550,NR_047599_exon_28_0_chrY_15591394_r,0,-,chrY,15591259,15591720,33


In [89]:
adata.write_h5ad('rna.h5ad')

In [90]:
adata

AnnData object with n_obs × n_vars = 100 × 1000
    varm: 'coord'

In [35]:
def groupby_agg(adata, granges):
    # coord = pr.PyRanges(adata.varm['coord'].reset_index())
    varm = adata.varm['coord'].copy()
    varm['idx'] = varm.index
    gb_gr_idx = bf.overlap(varm, granges, how='inner').groupby(['chrom_', 'start_', 'end_'])['idx'].apply(list).dropna()
    lst = []
    for i in gb_gr_idx:
        lst.append(adata[:, i].X.mean())
    return pd.Series(lst, index=gb_gr_idx.index)

In [34]:
varm = adata.varm['coord'].copy()
varm['idx'] = varm.index
# idx = bf.overlap(varm, gr, how='inner')['idx']
gb_gr_idx = bf.overlap(varm, gr, how='inner').groupby(['chrom_', 'start_', 'end_'])['idx'].apply(list).dropna()
lst = []
for i in gb_gr_idx:
    lst.append(adata[:, i].X.mean())
pd.Series(lst, index=gb_gr_idx.index)

chrom_  start_    end_    
chrX    584563    585326      0.9600
        1510501   1511838     0.9700
        1553851   1554115     0.7900
        2846195   2847511     1.0600
        10094050  10094406    1.1600
                               ...  
chrY    1363206   1363503     1.1100
        14532115  14533600    0.9200
        15591259  15591720    1.0725
        16941822  16942188    1.0500
        26979889  26980116    0.9700
Length: 72, dtype: float32

In [36]:
groupby_agg(adata, gr)

chrom_  start_    end_    
chrX    584563    585326      0.9600
        1510501   1511838     0.9700
        1553851   1554115     0.7900
        2846195   2847511     1.0600
        10094050  10094406    1.1600
                               ...  
chrY    1363206   1363503     1.1100
        14532115  14533600    0.9200
        15591259  15591720    1.0725
        16941822  16942188    1.0500
        26979889  26980116    0.9700
Length: 72, dtype: float32

In [59]:
bf.overlap(varm, gr, how='inner').groupby(['chrom_', 'start_', 'end_']).size()

chrom_  start_     end_     
chrX    155322     155553       0
                   245968       0
                   585326       0
                   1363503      0
                   1511838      0
                               ..
chrY    153990840  153046768    0
                   153070353    0
                   153285655    0
                   153600604    0
                   153991831    0
Length: 10368, dtype: int64

In [62]:
ab = bf.overlap(varm, gr, how='inner')

In [64]:
ab[ab['start_']==155322]

,chrom,start,end,Name,Score,Strand,idx,chrom_,start_,end_,CpG_
70,chrY,155399,155536,NR_028057_exon_3_0_chrY_155400_f,0,+,Gene_860,chrY,155322,155553,19
